# Homework 3

## Question 1: 

Modify the snps_density.py so that for each 1,000,000 bp region of each chromosome, the program calculates the percentage of SNPs that are transitions and the number of SNPs in each region. The results should be saved to a file called transitions.txt and look similar to this:

#### class SNP

In [18]:
## A class representing simple SNPs

class SNP:
    def __init__(self, chrname, pos, snpid, refallele, altallele):
        assert refallele != altallele, f"Error: ref == alt at pos {pos}"
        self.chrname = chrname
        self.pos = pos
        self.snpid = snpid
        self.refallele = refallele
        self.altallele = altallele


    ## Returns True if refallele/altallele is A/G, G/A, C/T, or T/C
    def is_transition(self):
        is_AG = (self.refallele == "A" and self.altallele == "G")
        is_GA = (self.refallele == "G" and self.altallele == "A")
        if is_AG or is_GA:
            return True  
        
        is_CT = (self.refallele == "C" and self.altallele == "T")
        is_TC = (self.refallele == "T" and self.altallele == "C")
        if is_CT or is_TC:
             return True

        return False

    ## Returns True if the snp is a transversion (ie, not a transition)
    def is_transversion(self):
        if self.is_transition():
            return False
        return True

In [2]:
## transition test; should not result in "Failed Test"

snp1 = SNP("1", 12351, "rs11345", "C", "T")
assert snp1.is_transition(), "Failed Test"

In [3]:
## transversion test; should not result in "Failed Test"
snp2 = SNP("1", 36642, "rs22541", "A", "T")
assert snp2.is_transversion(), "Failed Test"    ## Does not error

In [4]:
## error test; should result in "Error: ref == pos at position 69835"
snp3 = SNP("1", 69835, "rs53461", "A", "A")             ## Results in error

AssertionError: Error: ref == alt at pos 69835

#### Class Chromosome

In [74]:
class Chromosome:
    
    def __init__(self, chrname):
        self.chrname = chrname
        self.locations_to_snps = dict()

    ## Returns the chromosome name
    def get_name(self):
        return self.name

    
    ## Given all necessary information to add a new SNP, create
    ## a new SNP object and add it to the SNPs dictionary. If
    ## a SNP already exists at that location, or
    ## the given chrname doesn't match self.chrname, an error is reported.
    def add_snp(self, chrname, pos, snpid, refallele, altallele):
        ## If there is already an entry for that SNP, throw an error
        open_location = not (pos in self.locations_to_snps)
        assert open_location, f"Duplicate SNP: {self.chrname}:{pos}"
        
        ## If the chrname doesn't match self.chrname, throw and error
        assert chrname == self.chrname, "Chr name mismatch!"

        ## Otherwise, create the SNP object and add it to the dictionary
        newsnp = SNP(chrname, pos, snpid, refallele, altallele)
        self.locations_to_snps[pos] = newsnp


    ## Returns the number of transition snps stored in this chromosome
    def count_transitions(self):
        count = 0

        locations = self.locations_to_snps.keys()
        for location in locations:
            snp = self.locations_to_snps[location]
            if snp.is_transition():
                count = count + 1

        return count


    ## Returns the number of transversion snps stored in this chromosome
    def count_transversions(self):
        total_snps = len(self.locations_to_snps)
        return total_snps - self.count_transitions()

    
    ## returns the number of snps between l and m, divided by
    ## the size of the region
    def density_region(self, l, m):
        count = 0
        for location in self.locations_to_snps.keys():
            if location >= l and location <= m:
                count = count + 1

        size = m - l  + 1
        return 1000*count/float(size)


    ## returns the position of the last SNP known
    def get_last_snp_position(self):
        locations = list(self.locations_to_snps.keys())
        locations.sort()
        return locations[len(locations) - 1]
    

    ## given a region size, looks at non-overlapping windows
    ## of that size and returns a list of three elements for
    ## the region with the highest density:
    ## [density of region, start of region, end of region]
    def max_density(self, region_size):
        region_start = 1
        ## default answer if no SNPs exist [density, start, end]:
        best_answer = [0.0, 1, region_size - 1]
        
        ## todo: implement this method
        last_snp_position = self.get_last_snp_position()
        while region_start < last_snp_position:
            region_end = region_start + region_size - 1
            region_density = self.density_region(region_start, region_end)
            # if this region has a higher density than any we've seen so far:
            if region_density > best_answer[0]:
                best_answer = [region_density, region_start, region_end]
            
            region_start = region_start + region_size
        
        return best_answer
    
    
    ### Below are the new functions in the class "chromosome"


    # A function to count the number of snp, transition and transversion cout in a given region
    def region_trans(self, l, m):
        
        # initializing the variables
        count_snp = 0
        count_transition = 0
        count_transversion = 0
        
        # iterating through the dictionary (key = location, value = snp-related)
        for loc in self.locations_to_snps.keys():
            
            loc = int(loc)
            
            # when the location falls into the given region
            if loc >= l and loc <= m:
                
                # count the corresponding snp number, and transition / transversion count
                count_snp += 1
                if self.locations_to_snps[str(loc)].is_transition():
                    count_transition += 1
                if self.locations_to_snps[str(loc)].is_transversion():
                    count_transversion += 1
        
        # consider the exceptional cases where no snp in the region
        try:
            return [count_snp, round(count_transition / count_snp,3), round(count_transversion / count_snp,3)]
        except:
            return [count_snp, 0, 0]

        
    # A function to find the counts (snp, transition) in a chromosome, divided by specific region size
    
    def region_den(self, region_size):
        
        # lastsnp being the last snp in each chromosome
        lastsnp = int(sorted(self.locations_to_snps.keys(), reverse = True)[0])
        
        # initializing the output dictionary
        rden = dict()
        
        # iterating from the pos == 1 to the last snp, with each step == region size
        for loc in range(1, lastsnp, region_size):
            
            loc = int(loc)
            
            # calculate the counts for each region with specified region size
            reg_trans = self.region_trans(loc, loc + region_size - 1)
            
            # output with location, percentage of transition, and number of snp
            no_snp = reg_trans[0]
            percent_transition = reg_trans[1]
            key = str(loc) + ".." + str(loc + region_size - 1)
            rden[key] = [key, percent_transition, no_snp]
            
        return(rden)

#### Main program

In [72]:
import io

def main():

    # number of lines in file
    no = sum(1 for line in io.open("trio.sample.vcf"))

    with io.open("trio.sample.vcf") as fh:
        
        # initialize the dictionary; with keys = chromosome number; value = each snp
        chrnames_to_chrs = dict()
        
        # iterating through all the lines
        for i in range(no):
            
            # splitting each lines into lists
            ln = fh.readline().strip().split("\t")
            
            # skipping the first few lines which started with "#"
            if not ln[0].startswith("#"):
                
                # initialize a new chromosome if not found in the dict
                if ln[0] not in chrnames_to_chrs.keys():
                    chrnames_to_chrs[ln[0]] = Chromosome(ln[0])
                # adding the snp line by line
                chrnames_to_chrs[ln[0]].add_snp(ln[0],ln[1],ln[2],ln[3],ln[4])

    # writing the output file
    with open("transitions.txt", "w") as fh:
        
        # the column names
        fh.write("chromosome" + "\t" + "loc" + "\t" + "%transition" + "\t" + "num_snp" + "\n")
        
        # iterating through each chromosomes, e.g. 1,2,3...X
        for i in chrnames_to_chrs.keys():
            
            # calculate the counts (snp, transition) in a chromosome, divided by specific region size
            dc = chrnames_to_chrs[i].region_den(1000000)
            
            # for each region, write the data
            for loc in dc.keys():
                ls = dc[loc]
                fh.write(str(i) + "\t" + str(ls[0]) + "\t" + str(ls[1]) + "\t" + str(ls[2]) + "\n")

In [73]:
main()

## Question 2

The   module (used with     ) allows us to make random choices; for example, returns a random float between 0.0 and 1.0. The
function returns a random integer between a and b (inclusive); for example,
4) could return 1, 2, 3, or 4. There’s also a   function; given a list, it returns a single element (at random) from it. So, if   , then random.choice(bases) will return a single string, either "A", "C", "T", or "G".


In this program, define a Bug class. A bug object will represent an individual organism with a genome, from which a fitness can be calculated. For example, if a = Bug(), then a will have a self.genome which is a string of 100 random DNA bases (this should be generated in the constructor). You should implement a .get_fitness() method which returns a number computed from self.geome. For example, let’s simply define the fitness as
   Number of G bases + Number of C bases + 5 if “AAA” present in the genome.



Bug objects should also have a .mutate_random_base() method, which causes a random
random
  random.random()
random.choice()
element of implement a
In the
each run its
to be set to a random element from ["A", "C", "G", "T"]. Finally, method, which sets a specific index in the genome to a specific base:
should set self.genome[3] to "T".

In the main() on of your program, create a list of 10 Bug objects, and in a for-loop, have
method and print its new fitness.

In [2]:
import random

In [245]:
class bug:
    
    
    def __init__(self):
        
        # four bases
        self.bases = ["A", "T", "C", "G"]
        
        # generating 100 bases by random
        self.genome = [random.choice(self.bases) for i in range(100)]
        
        
    # function to mutate a random base
    def mutate_base(self):
        
        # generate a random number from 0 to 99
        base_no = random.randint(0,99)
        # mutate the base according to the above number
        original_base = self.genome[base_no]
        # ensure the base is mutated
        while self.genome[base_no] == original_base:
            self.genome[base_no] = random.choice(self.bases)
        return(self.genome)
        
        
    # function to set a base to another desired base
    def set_base(self, index, base):
        
        # ensure the desired base is not equal to the original base
        assert base != self.genome[index], "error: base already set"
        # set the base
        self.genome[index] = base
        return(self.genome)
        
        
    # function to get the fitness
    def get_fitness(self):
        
        # initializing
        self.fitness = 0
        fitness = 0
        
        # calculating the fitness by assessing the GC content
        for i in range(len(self.genome)):
            if self.genome[i] == "G" or self.genome[i] == "C":
                fitness += 1
        
        # joining the list of genome as a string
        genome_str = "".join(self.genome)
        # fitness + 5 if "AAA" is present
        if "AAA" in genome_str:
            fitness += 5
        self.fitness = fitness
        return(self.fitness)
        
    # Function to replace the entire genome 
    # for question 3
    def replace_genome(self, genomes):
        newgenomes = [i for i in genomes] # this is very important
        self.genome = newgenomes
        return(self.genome)

#### Testing the program

In [246]:
# initialization of a bug
a = bug()
print(a.genome)

['G', 'G', 'T', 'T', 'T', 'G', 'A', 'A', 'G', 'G', 'T', 'C', 'C', 'G', 'A', 'T', 'A', 'T', 'G', 'T', 'G', 'G', 'C', 'T', 'G', 'G', 'T', 'T', 'A', 'T', 'C', 'T', 'A', 'A', 'T', 'T', 'G', 'C', 'C', 'C', 'C', 'C', 'G', 'T', 'A', 'C', 'A', 'G', 'A', 'A', 'G', 'G', 'C', 'A', 'G', 'G', 'T', 'C', 'T', 'G', 'G', 'G', 'A', 'T', 'C', 'C', 'C', 'C', 'G', 'G', 'C', 'C', 'T', 'T', 'C', 'T', 'C', 'A', 'G', 'A', 'C', 'C', 'T', 'G', 'A', 'G', 'C', 'A', 'C', 'A', 'G', 'T', 'C', 'C', 'G', 'A', 'T', 'G', 'C', 'G']


In [247]:
# get fitness
a.get_fitness()
a.fitness

57

In [248]:
b = [i for i in a.genome]
# random mutation
a.mutate_base()

# checking which has been mutated

for i in range(len(b)):
    if b[i] != a.genome[i]:
        print(i)

13


In [249]:
c = [i for i in a.genome]
# set base
a.set_base(0, "T")

# checking if the base has been set

for i in range(len(c)):
    if c[i] != a.genome[i]:
        print(i)

0


## Question 3:

(Bonus) Write another program based on the code of previous question and simlulate the 
evolutonary progress. Frist, create a Population class. Population objects will have a list of Bug 
objects (say, 50) called self.bug_list. This Population class should have also
a .create_offspring() method, which will:

1) create an empty new_pop list, 
2) for each oldbug in self.bug_list: 
    a) create a new Bug object newbug, 
    b) set the genome of newbug to be the same as that of oldbug, 
    c) call newbug.mutate_random_base(), and 
    d) add oldbug and newbug to new_pop. 
3) set self.bug_list to new_pop.

The Population class will also have a .cull() method; this should reduce self.bug_list to the 
top 50% of bug objects by fitness. You might find .__lt__() in Bug class and similar methods 
useful, as they will allow you to sort self.bug_list by fitness if implemented properly.nFinally, 
implement a .get_mean_fitness() method in the Population class, which should return the 
average fitness of self.bug_list.

In the main() function of your program, first instantiate a p = Population() object and then in 
a for-loop of 20 iterations:

1) run p.create_offspring(), 
2) run p.cull(), and 
3) print p.get_mean_fitness(), allowing you to see the evolutionary progress of your 
simulation.

Your program should simulate the evolutionary progress, where population fitness is improving. See 
the example output below.

>Fitness of population during evolutionary progress

>(Assuming fitness = num_Gs + num_Cs (+ 5, if AAA present in genome)):

>53.36

>57.62

>59.96

>60.94

In [236]:
class population:
    
    # initializing 50 random bugs
    def __init__(self):
        self.bug_list = [bug() for i in range(50)]

        
    # create 50 offsprings
    def create_offspring(self):
        
        # initializing
        new_pop = []
        for i in self.bug_list:
            
            # create a new bug
            b = bug()
            
            # replace the genome of the new bug with the old bug
            b.replace_genome(i.genome)
            
            # replace the new bug genome
            b.mutate_base()
            
            # add the new bug to the new_pop list
            new_pop.append(b)
            
        # updating the bug_list
        self.bug_list = self.bug_list + new_pop
        
        
    # A function to reduce the bug list to the fittest 50%
    def cull(self):
        
        # initializing
        new_pop = list()
        fitness = dict()
        
        # calculating the fitness for each bug, appending the results into a list
        for i in self.bug_list:
            i.get_fitness()
            fitness[i] = i.fitness     
            
        # sorting the dict by value
        fitness = dict(sorted(fitness.items(), key = lambda i:i[1], reverse = True))
            
        # Selecting the first half of the population
        for i in range((len(self.bug_list)//2)):
            bug = list(fitness.keys())[i]
            new_pop.append(bug)
                    
        # final bug list
        self.bug_list = new_pop
        
        
    def mean_fitness(self):
        
        # module needed for calculating the mean
        import statistics
        
        # initializing
        fitness = list()
        
        # get the fitness for each bug
        for i in self.bug_list:
            i.get_fitness()
            fitness.append(i.fitness)
            
        # calculate the mean for all the bugs
        self.mean_fit = (statistics.mean(fitness))

In [243]:
# the main function

def main():
    
    p = population()
    
    print("Fitness of population during evolutionary progress: \n")
    
    print("(Assuming fitness = num_Gs + num_Cs (+ 5, if AAA present in genome)): \n")

    for i in range(20):
        
        p.create_offspring()
        p.cull()
        p.mean_fitness()
        print(f"Generation {i}: \t{p.mean_fit} \n")

In [244]:
main()

Fitness of population during evolutionary progress: 

(Assuming fitness = num_Gs + num_Cs (+ 5, if AAA present in genome)): 

Generation 0: 	56.06 

Generation 1: 	57.5 

Generation 2: 	58.92 

Generation 3: 	60.18 

Generation 4: 	61.3 

Generation 5: 	62.14 

Generation 6: 	63.28 

Generation 7: 	64.3 

Generation 8: 	65.3 

Generation 9: 	65.94 

Generation 10: 	66.64 

Generation 11: 	67.34 

Generation 12: 	67.78 

Generation 13: 	68.4 

Generation 14: 	68.88 

Generation 15: 	69.28 

Generation 16: 	69.54 

Generation 17: 	70.08 

Generation 18: 	70.44 

Generation 19: 	70.98 

